# Precontext

As a practitioner and enthusiast in simulation, I have always been fascinated by modeling business, economics, politics, and more through programs. In this notebook, I will explore the potential outcomes of integrating Mechanism Design theories with Generative AI by selectively revealing information to agents and observing their reactions.

Through repeated experiments, this approach achieves three key goals:

    1.Gaining a fresh perspective on GenAI models.
    2.Understanding how AI agents respond to economic theory-driven rules.
    3.Most importantly—finding enjoyment in the process (and by 'you,' I really mean 'I').

This is the first step in my exploration, and I’m making it public to invite discussion. If you share a similar interest, feel free to comment and exchange ideas.

Mechanism design (sometimes implementation theory or institution design)[1] is a branch of economics and game theory. It studies how to construct rules—called mechanisms or institutions—that produce good outcomes according to some predefined metric, even when the designer does not know the players' true preferences or what information they have. Mechanism design thus focuses on the study of solution concepts for a class of private-information games. 

For more information about Mechanism Design: https://en.wikipedia.org/wiki/Mechanism_design

# I. Config rules and information to be revealed to agents

In [1]:
delimiter = "####"
system_message = f"""
This is a bilateral trade game following VCG rules defined in VCG mechanism. \
In this game there are only two roles, buyer and seller. \
buyer and seller have their own evaluation for a good and that is their private information, \
in the meantime they also have declared evaluation. Both buyer and seller want to maximize their own utility. \
Remember, this is simotaneous game, buyer and seller give the declared evaluation at the same time and whether the trade would or wouldn't occur will be decided by that

Step 1:{delimiter} Think about what should be the declared evaluation

Step 2:{delimiter}: State your declared evaluation in the form of ##### Declared_Evaluation=Number #####
"""

In [2]:
buyer_private_valuation = 5  # Buyer knows their own valuation
seller_private_valuation = 3  # Seller knows their own cost

In [3]:
buyer_prompt = f"""
You are a buyer, your private evaluation for the good is {buyer_private_valuation}""".strip()
seller_prompt = f"""
You are a seller, your private evaluation for the good is {seller_private_valuation}""".strip()

In [4]:
buyer_model = 'gemini-2.0-flash-thinking-exp'
seller_model = 'gemini-2.0-flash-thinking-exp'

# II. Technical configuration

In [5]:
from kaggle_secrets import UserSecretsClient
from google import genai


user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini")

client = genai.Client(
    api_key=secret_value_0,
    http_options={'api_version': 'v1alpha'},
)

In [6]:
# pip install --upgrade google-generativeai
# ?client.models.generate_content

In [7]:


messages = [
    f"{system_message}",
    f"{buyer_prompt}",
]

response = client.models.generate_content(
    model=buyer_model,
    contents=messages,
)

# print(response.text)

buyer_thoughts = response.text

In [8]:


messages = [
    f"{system_message}",
    f"{seller_prompt}",
]

response = client.models.generate_content(
    model=seller_model,
    contents=messages,
)

# print(response.text)

seller_thoughts = response.text

In [9]:
import re

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", buyer_thoughts)

if match:
    buyer_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {buyer_declared}")
else:
    print("No match found")

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", seller_thoughts)

if match:
    seller_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {seller_declared}")
else:
    print("No match found")

Extracted number: 5
Extracted number: 3


# III. Conclusions

In [10]:
def vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation,declared_valuation,declared_cost, d=1):
    """Simulates bilateral trade with the VCG mechanism.

    Args:
        buyer_private_valuation: The buyer's true valuation.
        seller_private_valuation: The seller's true cost.
        d: Externality scaling factor.

    Returns:
        Tuple: (trade_occurs, price, buyer_utility, seller_utility)
    """
    # Buyer and seller use LLMs to decide their declared valuations
    buyer_declared_valuation = declared_valuation
    seller_declared_valuation = declared_cost

    # Seller's declared valuation with externality
    seller_declared_true_valuation = -d * seller_declared_valuation

    # Trade decision based on declared valuations
    if buyer_declared_valuation >= -seller_declared_true_valuation:
        trade_occurs = True
    else:
        trade_occurs = False

    if trade_occurs:
        # VCG price based on declared valuations
        price = -seller_declared_true_valuation

        # Buyer's utility (using private valuation)
        buyer_utility = buyer_private_valuation - price

        # Seller's transfer (using declared valuations)
        seller_transfer = d * buyer_declared_valuation - buyer_declared_valuation

        # Seller's utility (using private valuation)
        seller_utility = seller_transfer + (-d * seller_private_valuation)
    else:
        price = 0
        buyer_utility = 0
        seller_utility = -d * seller_private_valuation

    return (trade_occurs, price, buyer_utility, seller_utility, 
            buyer_declared_valuation, seller_declared_valuation)



results = vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation, buyer_declared, seller_declared)

trade_occurs, price, buyer_utility, seller_utility, \
buyer_declared_valuation, seller_declared_valuation = results

print(f"Buyer private valuation: {buyer_private_valuation} \n")
print(f"Buyer declared valuation: {buyer_declared_valuation} \n")
print(f"Buyer thoughts: \n {buyer_thoughts} \n")
print(f"Seller private valuation (inherent cost): {seller_private_valuation} \n")
print(f"Seller declared valuation (inherent cost): {seller_declared_valuation} \n")
print(f"Buyer thoughts: \n {seller_thoughts} \n")

Buyer private valuation: 5 

Buyer declared valuation: 5 

Buyer thoughts: 
 Step 1:#### Think about what should be the declared evaluation

In a VCG mechanism, for bilateral trade, the dominant strategy for both the buyer and the seller is to declare their true valuation. Let's analyze why this is the case for a buyer.

* **Scenario 1: Trade occurs (Declared_Buyer_Value >= Declared_Seller_Value)**
    * The buyer's utility is Buyer's_True_Value - Declared_Seller_Value.
    * The buyer wants trade to occur if and only if Buyer's_True_Value >= Seller's_True_Value.
    * In VCG, trade happens if Declared_Buyer_Value >= Declared_Seller_Value.  The price paid by the buyer is Declared_Seller_Value.

* **Scenario 2: Trade does not occur (Declared_Buyer_Value < Declared_Seller_Value)**
    * The buyer's utility is 0.
    * The buyer wants to avoid trade if Buyer's_True_Value < Seller's_True_Value.

Let's consider different strategies for the buyer, assuming the buyer's true valuation is 5.

*

In [11]:
print(f"Trade occurs: {trade_occurs} \n")
if trade_occurs:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")
else:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")

Trade occurs: True 

Price: 3
Buyer utility: 2
Seller utility: -3


# X. References

https://ai.google.dev/gemini-api/docs/thinking